# Optimisation de la planification chez Ciment quebec

## Objective
    - Schedule the daily concrete deliveries
    - Minimize the vehicle travel times
    - Minimize waiting time at loading station ?



## Constraints

- Concrete is produced at several *homogeneous* production sites located some distance away from the customers. 
- Each truck has its assigned production site which is its start depot
- At the end of the day, a truck returns to its assigned depot

### Notation
- $\mathcal{O}$: set of orders 
- $\mathcal{P}$: set of depots 
- $\mathcal{K}_p$ : set of trucks assigned to depot $p$
- $Q_o$: Demand of customer order $o$
- $Q^k$: Capacity of a truck
- $R^p$: Capacity of a depot $p$
- $p_k$: home plant of truck k
-  $[E_o, F_o]$: Time window associated with order o
- $T^k_{o1,o2}$: time to go from the construction site of order $o_1$ to the construction site of order $o_2$ while also being loaded at the closest plant enroute, from which the construction site of order $o_2$ can be reached in less than 2 hours; does not depend on the amount of concrete in the truck
- $T_{p,o}$ : time to go from the location site of plant $p$ to the construction site of order $o$, 
- $T_{o,p}$ :  time to go from the construction site of order $o$ to the location site of plant $p$.
- $L^k_o$: time to load truck $k$ at the construction site $o$, depends on the truck and on the amount of loaded concrete.
- $U^k_o$: time to unload truck $k$ at the construction site $o$, depends on the truck and the construction site, but not on the amount of unloaded concrete.
- $\mathcal{D}_o$: set of potential deliveries associated with each order, index of the sequence of visiting trucks of $o$.
- |$\mathcal{D}_o$|: upper bound of the approximate number of deliveries for an order $o$.

### Bound of number of deliveries
#### The minimum number of deliveries required 
- Assume that all unloading operations are executed by the largest truck available.
- $LB_o =  \lceil { \frac{Q_o}{max \{ Q^k\}} } \rceil$
#### The maximum number of deliveries required 
- Assume that all unloading operations are executed by the smallest truck available.
- $UB_o =  \lceil { \frac{Q_o}{min \{ Q^k\}} } \rceil$


## Decision variables

- $x^k_{o_1,d_1;o_2,d_2}=1$ when truck $k$ serves delivery $d_2$ of order $o_2$ immediately after serving delivery $d_1$ of order $o_1$.
- $x^k_{d_1d_2p}=1$ when truck $k$ serves delivery $d_2$  immediately after serving delivery $d_1$ from plant $p$
- $x^k_{ij}=1$ when truck $k$ travels arc ($i$, $j$)

Plants are capacitated.

- $ \sum_{k \in \mathcal{K}} \sum_{d \in \delta^{+}(p)}{q_d x^k_{pd}} \leq R_p $ for delivery $d$ and depot $p$

Trucks are capacitated

- $ \sum_{(i,o) \in A}{q_ox^k_{io}} \leq Q^k $

-  $u^k_{o,d}=1$  iif the first delivery to be performed by truck $k$ along the planning horizon (day) refers to executing delivery $d$ of order $o$.
-  $v^k_{o,d} =1 $ iif delivery $d$ of order $o$ is truck $k$ last assignment before returning to its home plant.

- $y^k_{o,d}=1$ iif  truck $k$ executes delivery $d$ of order $o$. 
- $y^k_{d}=1$ iif  truck $k$ executes delivery $d$

- $z_{o,d}=1$ if the d-th potential delivery associated with order $o$ is executed. 

Its  serves as an indicator whether or not a certain delivery $d$ associated with order $o$ is supposed to be executed. 

Recall that $d \in \mathcal{D}_o$ is simply an index, so $d = 1$ represents the first delivery, that must be executed, and $d = |\mathcal{D}_o|$ represents a true or dummy delivery depending if it is executed or not, respectively

- $a_{o,d}$ ($b_{o,d}$) refers to the start (end) of the unloading operation associated with delivery $d$ of order $o$.

    - $b_{o,d}$ coincides with the time when a truck leaves $o$ if and only if $o \notin \mathcal{O}'$ or $d > 1$ or $Q_k \geq Q_o$.
    - $a_{o,d}$ coincides with the time when the truck assigned to this particular delivery starts its unloading operation at the construction site corresponding to order $o$.

- $l_o$ captures any delayed start of the first delivery associated with order $o$. In case the first delivery starts after the end of the given time window interval $[E_o, F_o]$, $l_o$ evaluates the length of the resulting gap. In case the first delivery starts within the time window it will be equal to 0. Starting the very first delivery before $E_o$ is not permitted.


## Objective function

 - Minimize the total time traveled by trucks. 
 - Additionally gaps between consecutive deliveries, or starting the first delivery of any order o after $F_o$  are not desirable and need to be penalized. 
 - To combine these features in a single objective function, we make use of the parameter $\beta$ for penalizing gaps between consecutive deliveries

 min $\sum_{o_1,o_2 ∈ \mathcal{O},\\ d_1∈ \mathcal{D}_{o_1} \\ d_2 ∈ \mathcal{D}_{o_1} \\ k∈ \mathcal{K}} {T^k_{o_1,o_2} · x^k_{o_1,d_1;o_2,d_2}}$ +  $\sum_{o ∈ \mathcal{O},\\ d ∈ \mathcal{D}_{o} \\ k∈ \mathcal{K}} {T_{p_k,o} · u^k_{o,d}}$ +  $\sum_{o ∈ \mathcal{O},\\ d ∈ \mathcal{D}_{o} \\ k∈ \mathcal{K}} {T_{o,p_k} · v^k_{o,d}}$ +  $\beta \sum_{o ∈ \mathcal{O}}\left(l_o + \sum_{d \in \mathcal{D}_0}(a_{o,d}- b_{o,d-1})\right) $



- Reduce symmetric solutions: the very first delivery associated with any order needs to be executed by all means:
    - $z_{o,1}= 1$


- Only consecutive deliveries may need to be executed
    - $z_{o,d} \leq z_{o,d−1}$  $∀o ∈ \mathcal{O}, d ∈ \mathcal{D}_o : d>1$.


- When delivery $d$ of order $o$ is decided to be executed, a truck k must be assigned to it:
    - $ \sum_{k \in \mathcal{K}}y^k_{o,d} = z_{o,d} $ $∀o ∈ \mathcal{O}, d \in \mathcal{D}_o :  $ d>1$

- If truck $k$ is assigned to the execution of delivery $d$ associated with order $o$, the truck needs to go to the construction site of $o$:
    - $y^k_{o,d} = u^k_{o,d} + \sum_{ o_1 \in \mathcal{O} \backslash \{o\} \\ d_1 \in \mathcal{D}_o } {x^k_{o_1,d_1;o,d}} + \sum_{d_1 \in \mathcal{O}_o: \\ d_1 < d }{x^k_{o,d_1;o,d}} $ $\forall o \in \mathcal{O}, d \in \mathcal{D}_o, k \in \mathcal{K} $


- After executing a delivery, a truck must go somewhere else
    - $y^k_{o,d} = v^k_{o,d} + \sum_{ o_2 \in \mathcal{O} \backslash \{o\} \\ d_2 \in \mathcal{D}_o } {x^k_{o,d;o_2,d_2}} + \sum_{d_2 \in \mathcal{O}_o:\\ d_2 > d }{x^k_{o,d;o,d_2}} $ $\forall o \in \mathcal{O}, d \in \mathcal{D}_o, k \in \mathcal{K} $

-  A truck is allowed to leave its home plant (and execute its very first unloading operation per day) at most once:
    - $ \sum_{ o \in \mathcal{O} \\ d \in \mathcal{D}_o }{u^k_{o,d} \leq 1} $ $ k \in \mathcal{K} $

- In case a truck leaves its home plant it also needs to return there by the end of the day:
    - $ \sum_{ o \in \mathcal{O} \\ d \in \mathcal{D}_o } {u^k_{o,d}} = \sum_{ o \in \mathcal{O} \\ d \in \mathcal{D}_o }{v^k_{o,d}} $ $ k \in \mathcal{K} $

- The time difference between two consecutive unloading operations assigned to the same truck needs to be big enough to allow the truck driving to the closest plant and being loaded there.
    - $  a_{o_2,d_2} \geq b_{o_1,d_1} + T^k_{o_1,o_2}.x^k_{o_1,d_1;o_2;d_2} - M(1- x^k_{o_1,d_1;o_2,d_2}) $
    
     $ \forall o_1, o_2 \in \mathcal{O}, o_1 \neq o_2, d_1 \in \mathcal{D}_{o_1}, d_2 \in \mathcal{D}_{o_2}, k \in \mathcal{K}:$  $d1 > 1$ or $Q^k \geq Q_{o_1}   $

     - For two identical orders:
        -  $a_{o,d_2} \geq b_{o,d_1} + T^k_{o,o}.x^k_{o,d_1;o;d_2} $ 
        $ \forall o_1, o_2 \in \mathcal{O}, d_1, d_2 \in \mathcal{D}_{o_1}, d_1 < d_2 $  

    - M is defined as the difference between the latest possible time when truck $k$ could leave the construction site associated with order $o_1$ after having executed delivery $d_1$ and the earliest possible start of delivery $d_2$ for order $o_2$.

- Two trucks cannot be unloaded at the same time in a construction site:
    - $a_{o,d } \geq b_{o,d−1}$ $ \forall o \in \mathcal{O}, d \in \mathcal{D}_o : d>1$

 - The first delivery of any order has to start after the begin of the given time window:
      - $a_{o,1} \geq E_o$  $ \forall o \in \mathcal{O}$

- A late start of the first delivery of order $o$ needs to be penalized in the objective function accordingly. The first delivery of any order is considered as starting late in case the unloading operation is initiated after the end of the corresponding time window:
    - $l_o \geq a_{o,1} - F_o $

- To ensure that the required demand is totally delivered, the cumulative capacity of all trucks serving an order may not be smaller than such demand:
    - $ \sum_{d \in \mathcal{D}_o \\ k \in \mathcal{K} }{Q^ky^k_{o,d}} \geq Q_o  $

 - The time to execute any delivery is determined as a constant dependent on the truck and the construction site:
    - $b_{o,d} − a_{o,d}  \geq U^k_o · y^k_{o,d}$ $∀o ∈ O, d ∈ \mathcal{D}_o, k ∈ \mathcal{K}$
   - $b_{o,d} − a_{o,d} \leq U^k_o + M'(1 − y^k_{o,d})$ $∀o ∈ O, d ∈ \mathcal{D}_o , k ∈ \mathcal{K}$, $M' = max\{U^k_o : k ∈ K\}− U^k_o$.

In [ ]:
## Formulation

   
   
 
   



In [ ]:
Nature du produit
Plusieurs types de béton
Différents temps de chargement
Différents temps de déchargement
Opération de refroidissement 
Déplacement sur le chantier
Attente sur le chantier pour le déchargement
Multiples voyages pour servir le client
Livraison à une heure donnée  ( Pas avant pas après)


### Qu'est-ce que j'ai?
- Des livreurs
    - dans des camions
    - qui transportent du beton
    - vers des chantiers
#### Je connais:
- Les horaires des livreurs
- Les positions des sites de chargement de dépôt
- Les positions des chantiers de construction
- La quantité demandée par client
- La fenêtre de temps de livraison


### Qu'est ce que je cherche ?
    - Construire les itinéraires de chaque livreur disponible
    - Objectif:
        - Minimiser les temps de livraison
        - Maximiser le 
    